In [1]:
import findspark
findspark.init('/usr/local/spark')

In [2]:
import pyspark

In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName("joins_vsbroadcasting").getOrCreate()

In [ ]:
spark

In [5]:
cust_df = spark.read.csv('customers.txt', sep='\t', header=True, inferSchema=True)

In [21]:
cust_df.show(5)

+-----------+--------------+-------------+--------------+----------------+
|customer_id| customer_name|customer_city|customer_state|customer_zipcode|
+-----------+--------------+-------------+--------------+----------------+
|      11039|   Mary Torres|       Caguas|            PR|             725|
|       5623|    Jose Haley|     Columbus|            OH|           43207|
|       5829|    Mary Smith|      Houston|            TX|           77015|
|       6336|Richard Maddox|       Caguas|            PR|             725|
|       1708|Margaret Booth|    Arlington|            TX|           76010|
+-----------+--------------+-------------+--------------+----------------+
only showing top 5 rows



In [ ]:
cust_df.printSchema()

In [6]:
sales_df = spark.read.csv("salestxns.tsv", sep='\t', inferSchema=True)

In [ ]:
sales_df1.show(2)

In [7]:
sales_df1 = sales_df.toDF("TxnId","CategoryId","CategoryName","ProductId","ProductName","Price","Quantity","customer_id")

In [8]:
sales_df1.printSchema()

root
 |-- TxnId: integer (nullable = true)
 |-- CategoryId: integer (nullable = true)
 |-- CategoryName: string (nullable = true)
 |-- ProductId: integer (nullable = true)
 |-- ProductName: string (nullable = true)
 |-- Price: double (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- customer_id: integer (nullable = true)



In [9]:
cust_df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_name: string (nullable = true)
 |-- customer_city: string (nullable = true)
 |-- customer_state: string (nullable = true)
 |-- customer_zipcode: integer (nullable = true)



In [ ]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType

In [ ]:
# Customised_schema = StructType([StructField("TxnID", IntegerType(), True),\
# StructField("Category_ID", IntegerType(), True]) \ .....

In [ ]:
sales_df1.show(4)

In [10]:
spark.conf.get("spark.sql.autoBroadcastJoinThreshold")

'10485760'

In [11]:
10485760/1024

10240.0

In [12]:
10240.0/1024

10.0

In [ ]:
# spark.conf.set("spark.sql.autoBroadcastJoinThreshold", 50*1024*1024)

In [14]:
spark.conf.set("spark.sql.autoBroadcastJoinThreshold",-1)

In [15]:
import datetime

In [16]:
phys_exec_plan = sales_df1.join(cust_df, on = 'customer_id',how='left').explain()
start_time = datetime.datetime.now()
sales_df1.join(cust_df, on = 'customer_id',how='left').count()
end_time = datetime.datetime.now()
print("Processing time taken : ", end_time-start_time)

== Physical Plan ==
*Project [customer_id#59, TxnId#52, CategoryId#53, CategoryName#54, ProductId#55, ProductName#56, Price#57, Quantity#58, customer_name#13, customer_city#14, customer_state#15, customer_zipcode#16]
+- SortMergeJoin [customer_id#59], [customer_id#12], LeftOuter
   :- *Sort [customer_id#59 ASC NULLS FIRST], false, 0
   :  +- Exchange hashpartitioning(customer_id#59, 200)
   :     +- *Project [_c0#35 AS TxnId#52, _c1#36 AS CategoryId#53, _c2#37 AS CategoryName#54, _c3#38 AS ProductId#55, _c4#39 AS ProductName#56, _c5#40 AS Price#57, _c6#41 AS Quantity#58, _c7#42 AS customer_id#59]
   :        +- *FileScan csv [_c0#35,_c1#36,_c2#37,_c3#38,_c4#39,_c5#40,_c6#41,_c7#42] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/home/hduser/del_hyd/salestxns.tsv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<_c0:int,_c1:int,_c2:string,_c3:int,_c4:string,_c5:double,_c6:int,_c7:int>
   +- *Sort [customer_id#12 ASC NULLS FIRST], false, 0
      +- Exchange has

In [17]:
spark.conf.set("spark.sql.autoBroadcastJoinThreshold",10485760)

In [18]:
from pyspark.sql.functions import broadcast

In [19]:
phys_exec_plan = sales_df1.join(broadcast(cust_df), on = 'customer_id',how='left').explain()
start_time = datetime.datetime.now()
sales_df1.join(broadcast(cust_df), on = 'customer_id',how='left').count()
end_time = datetime.datetime.now()
print("Processing time taken : ", end_time-start_time)

== Physical Plan ==
*Project [customer_id#59, TxnId#52, CategoryId#53, CategoryName#54, ProductId#55, ProductName#56, Price#57, Quantity#58, customer_name#13, customer_city#14, customer_state#15, customer_zipcode#16]
+- *BroadcastHashJoin [customer_id#59], [customer_id#12], LeftOuter, BuildRight
   :- *Project [_c0#35 AS TxnId#52, _c1#36 AS CategoryId#53, _c2#37 AS CategoryName#54, _c3#38 AS ProductId#55, _c4#39 AS ProductName#56, _c5#40 AS Price#57, _c6#41 AS Quantity#58, _c7#42 AS customer_id#59]
   :  +- *FileScan csv [_c0#35,_c1#36,_c2#37,_c3#38,_c4#39,_c5#40,_c6#41,_c7#42] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/home/hduser/del_hyd/salestxns.tsv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<_c0:int,_c1:int,_c2:string,_c3:int,_c4:string,_c5:double,_c6:int,_c7:int>
   +- BroadcastExchange HashedRelationBroadcastMode(List(cast(input[0, int, true] as bigint)))
      +- *FileScan csv [customer_id#12,customer_name#13,customer_city#14,customer_state

In [ ]:
without_broadcast = 1.17
with_broadcast = 0.11
process_time_reduced = without_broadcast-with_broadcast
pct = process_time_reduced/without_broadcast
print("The process time reduced with the help of broadcasting is :", pct)